<a href="https://colab.research.google.com/github/linqus/nlp-huggingface/blob/main/notebooks/unit7/nlp_7_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install datasets transformers[sentencepiece] sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 7.8 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

raw_datasets = load_dataset('kde4', lang1='en', lang2='fr')

raw_datasets

Generating train split:   0%|          | 0/210173 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

In [82]:
split_datasets = raw_datasets['train'].train_test_split(test_size=0.1, shuffle=True, seed=20)
split_datasets['validation'] = split_datasets.pop('test')
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [12]:
split_datasets['train'][1]

{'id': '152754',
 'translation': {'en': 'Default to expanded threads',
  'fr': 'Par défaut, développer les fils de discussion'}}

In [15]:
from transformers import pipeline

model_checkpoint = 'Helsinki-NLP/opus-mt-en-fr'

translator = pipeline('translation', model_checkpoint)
outputs = translator(split_datasets['train'][1]['translation']['en'])
outputs

[{'translation_text': 'Par défaut pour les threads élargis'}]

In [43]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors='pt')

en_inputs = tokenizer(split_datasets['train'][1]['translation']['en'])
en_inputs

{'input_ids': [47591, 12, 9842, 19634, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [44]:
wrong_inputs = tokenizer(split_datasets['train'][1]['translation']['fr'])


In [45]:
full_inputs = tokenizer(split_datasets['train'][1]['translation']['en'], text_target=split_datasets['train'][1]['translation']['fr'])
full_inputs

{'input_ids': [47591, 12, 9842, 19634, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]}

In [46]:
print('wrong fr: ', tokenizer.convert_ids_to_tokens(wrong_inputs['input_ids']))
print('good fr: ', tokenizer.convert_ids_to_tokens(full_inputs['labels']))

wrong fr:  ['▁Par', '▁dé', 'f', 'aut', ',', '▁dé', 've', 'lop', 'per', '▁les', '▁fil', 's', '▁de', '▁discussion', '</s>']
good fr:  ['▁Par', '▁défaut', ',', '▁développer', '▁les', '▁fils', '▁de', '▁discussion', '</s>']


In [73]:
max_length=128

def process_sentences(examples):

  en_sentences = [ex['en'] for ex in examples['translation']]
  fr_sentences = [ex['fr'] for ex in examples['translation']]
  result = tokenizer(en_sentences, text_target=fr_sentences, max_length=128, truncation=True)

  return result

In [77]:
samples = [split_datasets['train'][i] for i in range(3)]
samples = split_datasets['train'][:3]
samples

{'id': ['92924', '152754', '164335'],
 'translation': [{'en': "Calibration is about to check the value range your device delivers. Please move axis %1 %2 on your device to the maximum position. Press any button on the device or click on the'Next 'button to continue with the next step.",
   'fr': "Le calibrage va vérifier la plage de valeurs que votre matériel produit. Veuillez déplacer l'axe %1 %2 de votre périphérique à la position maximale. Appuyez sur n'importe quel bouton du périphérique ou sur le bouton « & #160; Suivant & #160; » pour la prochaine étape."},
  {'en': 'Default to expanded threads',
   'fr': 'Par défaut, développer les fils de discussion'},
  {'en': '*. ui *. UI_BAR_User Interface Files',
   'fr': '*. ui *. UI_BAR_Fichiers interface utilisateur'}]}

In [75]:
process_senteces(samples)

{'input_ids': [[34378, 226, 5783, 32, 200, 12, 3647, 4, 1223, 1628, 117, 4923, 23608, 3, 1789, 2942, 20059, 301, 548, 301, 331, 30, 117, 4923, 12, 4, 1528, 668, 3, 5734, 212, 9319, 30, 4, 4923, 57, 5487, 30, 4, 6, 32712, 25, 7243, 1160, 12, 621, 42, 4, 1156, 3009, 3, 0], [47591, 12, 9842, 19634, 9, 0], [1211, 3, 49, 9409, 1211, 3, 29140, 817, 3124, 817, 28149, 139, 33712, 25218, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[60, 7418, 5244, 8234, 740, 4993, 8, 6471, 5, 2218, 29, 193, 2220, 742, 3, 4366, 14237, 14, 6, 16600, 301, 548, 301, 331, 5, 193, 24275, 17, 8, 668, 6142, 3, 33640, 36, 81, 6, 5411, 2709, 9376, 22, 24275, 59, 36, 19, 9376, 153, 402, 29033, 13774, 402, 29033, 416, 27, 8, 4034, 4888, 3, 0], [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0], [1211, 3, 49, 9409, 121

In [88]:
tokenized_datasets = split_datasets.map(process_sentences, batched=True, remove_columns=['id', 'translation'])
tokenized_datasets

Map:   0%|          | 0/189155 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 21018
    })
})